# Connecting

In [1]:
import os

# setting working dir
os.chdir("../src")

In [2]:
from elasticsearch_db.elasticsearch import elastic_conection
from elasticsearch_db.elasticsearch import get_nlp_model

es = elastic_conection()

In [3]:
workspace_id = "dc1e7b3d-9137-4a20-a99c-d0d2029ef170"
workspace, exist = get_nlp_model(es, workspace_id=workspace_id)
exist

True

# Data Sourcing

In [4]:
from data_pipeline.data_sourcing import get_data

get_data?

Signature: get_data(workspace, es)
Docstring:
Get all the intents in the elasticsearch
workspace's index.

Parameters
----------
workspace: the workspace object
es: the elasticsearch conection instance

Returns
-------
list: a list of namedtuple Intents
    
Examples
--------

Raises
------

Notes
-----
File:      /usr/omni_nlu_api/src/data_pipeline/data_sourcing.py
Type:      function


In [5]:
data = get_data(workspace=workspace, es=es)
print(len(data))
print(data[0])

206
Intent(intent_name='Inativar_Posição', text_examples=['Como realizo a inativação de uma posição de minha estrutura?', 'Como realizo a reativação de uma posição em minha estrutura?', 'Em quanto tempo a inativação de uma posição é efetivada?', 'Gostaria de fazer a inativação de uma posição, como faço?', 'Realizei a inativação de uma posição e ela continua visível?'])


# Data Preprocessing

### Create artificial Examples

In [6]:
from data_pipeline.data_preprocessing import fill_missing_examples

fill_missing_examples?

Signature: fill_missing_examples(data, random_seed=42)
Docstring:
We expect at least 8 examples for each intent,
5 to train and 3 to test the ML models.

This function identify intents with less then 8 examples
and fill it up with artificial examples created from the 
exesting ones.

Parameters
----------
data: list of namedtuple representing Intents

Returns
-------
list: list of namedtuple representing Intents
    
Examples
--------
input = [
    Intent(
        intent_name='Inativar_Posição', 
        text_examples=['Como realizo a inativação de uma posição de minha estrutura?', 
                       'Como realizo a reativação de uma posição em minha estrutura?', 
                       'Em quanto tempo a inativação de uma posição é efetivada?', 
                       'Gostaria de fazer a inativação de uma posição, como faço?', 
                       'Realizei a inativação de uma posição e ela continua visível?']
        )
]

output = [
    Intent(
        intent_name='Inativar_

In [7]:
data = fill_missing_examples(data)

# Data Splitting